In [ ]:
# Setup, Version check and Common imports

# Python ≥3.8 is required
import sys
assert sys.version_info >= (3, 5)


# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

from tensorflow import keras
from tensorflow.keras import layers

# to make this notebook's output stable across runs
np.random.seed(42)

import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

print('Python version: ', sys.version_info)
print('TF version: ', tf.__version__)
print('Keras version: ', keras.__version__)
print('GPU is', 'available' if tf.config.list_physical_devices('GPU') else 'NOT AVAILABLE')


**0. Relevant Dataset Settings**

In [ ]:
# Example from Keras tutorial: https://keras.io/examples/nlp/addition_rnn/

# Dataset with TRAINING_SIZE examples
TRAINING_SIZE = 20000

# Maximum length of each operand is DIGITS.
# Each example corresponds to a simple addition between two integer values, with digits between 1 and DIGITS
DIGITS = 3

# If REVERSE = True, then the input sequence is reversed
REVERSE = False


# Maximum length of input is 'int + int' (e.g., '345+678').
MAXLEN = DIGITS + 1 + DIGITS

# Maximum length of output is DIGITS + 1


**1. Dataset Creation and Processing**

In [ ]:
# Definition of the class that will generate dataset examples

# Characters are encoded as a one-hot integer representation

class CharacterTable:
    # Given a set of characters:
    # Encode them to a one-hot integer representation
    # Decode the one-hot or integer representation to their character output
    # Decode a vector of probabilities to their character output

    def __init__(self, chars):
        # Initialize character table.
        # Arguments
        # chars: Characters that can appear in the input.

        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        # One-hot encode given string C.
        # Arguments
        # C: string, to be encoded.
        # num_rows: Number of rows in the returned one-hot encoding. This is used to keep the # of rows for each data the same.

        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        # Decode the given vector or 2D array to their character output.
        # Arguments
        # x: A vector or a 2D array of probabilities or one-hot representations;
        # or a vector of character indices (used with `calc_argmax=False`).
        # calc_argmax: Whether to find the character index with maximum probability, defaults to `True`.

        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[x] for x in x)


# All the numbers, plus sign and space for padding.
chars = "0123456789+ "
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print("Generating data...")
while len(questions) < TRAINING_SIZE:
    f = lambda: int(
        "".join(
            np.random.choice(list("0123456789"))
            for i in range(np.random.randint(1, DIGITS + 1))
        )
    )
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = "{}+{}".format(a, b)
    query = q + " " * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += " " * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print("Total questions:", len(questions))


In [ ]:
 # Visualize a few examples

 data = np.random.randint(0, len(questions)-10)

 for i in range(10):
   print(questions[data + i] + ' = ' + expected[data + i])

In [ ]:
# Data vectorization

print("Vectorization...")
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

#Create train and validation datasets
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print("Training Data:")
print(x_train.shape)
print(y_train.shape)

print("Validation Data:")
print(x_val.shape)
print(y_val.shape)


**2. Deep Learning Model**

In [ ]:
# Build the model
# See the worksheet for details

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

inputs = keras.Input(shape=[MAXLEN, len(chars)])
x = keras.layers.LSTM(128, return_sequences = False)(inputs)
x = layers.RepeatVector(DIGITS + 1)(x)
x = layers.LSTM(128, return_sequences=True)(x)
output = layers.TimeDistributed(layers.Dense(len(chars), activation="softmax"))(x)

model = keras.Model(inputs, output)



**3. Model Compilation, Training and Evaluation**

In [ ]:

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

In [ ]:
# Model training
# In each epoch, a sample of 10 validation examples is presented

epochs = 15
batch_size = 32


# Train the model each generation and show predictions against the validation dataset.
for epoch in range(1, epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize errors.
    total = 0
    error = 0
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        error += abs(int(correct)-int(guess))
        if correct == guess:
            total += 1
            print("☑ " + guess)
        else:
            print("☒ " + guess)
    print('Correct:', total/10, '(', total, '/10)')
    print('MAE: ', error/10)



In [ ]:
# Evaluate final performance on the validation set

model.evaluate(x_val, y_val)
